In [1]:
import requests
import json
import re
from tqdm.notebook import tqdm
import time
import glob
import pandas as pd
import redis
import numpy as np
import threading

In [2]:
api_key = "911907964c0d70dbf619caf7b08b67d1"

In [3]:
year = 2018
search = f'PUBYEAR = {year} AND NOT AFFIL (Chulalongkorn University) '


res = requests.get("https://api.elsevier.com/content/search/scopus",headers={"Accept" : "application/json","X-ELS-APIKey" : api_key},params={"query" : search,"cursor" : "*","count" : "5","view" : "STANDARD","sort" : "-citedby-count"})


In [4]:
res.json()

{'search-results': {'opensearch:totalResults': '3340554',
  'opensearch:itemsPerPage': '5',
  'opensearch:Query': {'@role': 'request',
   '@searchTerms': 'PUBYEAR = 2018 AND NOT AFFIL (Chulalongkorn University) ',
   '@startPage': ''},
  'cursor': {'@current': '*', '@next': 'AoJb6wYyMi1zMi4wLTg1MDU0Mjg3MzY1'},
  'link': [{'@_fa': 'true',
    '@ref': 'self',
    '@href': 'https://api.elsevier.com/content/search/scopus?cursor=*&count=5&query=PUBYEAR+%3D+2018+AND+NOT+AFFIL+%28Chulalongkorn+University%29+&view=STANDARD&sort=-citedby-count',
    '@type': 'application/json'},
   {'@_fa': 'true',
    '@ref': 'first',
    '@href': 'https://api.elsevier.com/content/search/scopus?cursor=*&count=5&query=PUBYEAR+%3D+2018+AND+NOT+AFFIL+%28Chulalongkorn+University%29+&view=STANDARD&sort=-citedby-count',
    '@type': 'application/json'},
   {'@_fa': 'true',
    '@ref': 'next',
    '@href': 'https://api.elsevier.com/content/search/scopus?cursor=AoJb6wYyMi1zMi4wLTg1MDU0Mjg3MzY1&count=5&query=PUBYEAR+%3

In [2]:
class MyQuery:
    def __init__(self,dir = "output",dataPerYear = 1600,offsetYear = 0):
        self.apiKey = input("Input your API key")
        self.start_year = 2018 + offsetYear
        self.end_year = 2024
        self.total_count = dataPerYear
        self.dir = dir
        self.cache = set([re.sub(f"{self.dir}\\\\","",e) for e in glob.glob(f"{self.dir}/*")])
    
    def _search_loop(self,year):
        search = f'PUBYEAR = {year} AND NOT AFFIL (Chulalongkorn University) '
        res = requests.get("https://api.elsevier.com/content/search/scopus",headers={"Accept" : "application/json","X-ELS-APIKey" : api_key},params={"query" : search,"cursor" : "*","count" : "5","view" : "STANDARD","sort" : "-citedby-count"})
        for c in tqdm(range(0,self.total_count,5)):
            jfile = res.json()
            for entry in jfile["search-results"]["entry"]:
                scopus_id = self.get_scopusId(entry['dc:identifier'])
                if (scopus_id in self.cache):
                    continue
                json_abstract = self.query_abstract(scopus_id)
                self.save_abstract(json_abstract,scopus_id)
            time.sleep(1)
            next_cursor = jfile["search-results"]['cursor']['@next']
            res = requests.get("https://api.elsevier.com/content/search/scopus",headers={"Accept" : "application/json","X-ELS-APIKey" : api_key},params={"query" : search,"cursor" : next_cursor,"count" : "5","view" : "STANDARD","sort" : "-citedby-count"})
        
    
    def search_loop(self):
        for year in tqdm(range(self.start_year,self.end_year)):
            self._search_loop(year)
        return
        
    def get_scopusId(self,identifier):
        return re.sub("SCOPUS_ID:","",identifier)
        
    def query_abstract(self,scopus_id):
        res = requests.get(f"https://api.elsevier.com/content/abstract/scopus_id/{scopus_id}",headers={"Accept" : "application/json","X-ELS-APIKey" : self.apiKey},params={"view" : "FULL"})
        return res.json()
    
    def save_abstract(self,json_abstract,scopus_id):
        with open(f"{self.dir}/{scopus_id}","w") as file:
            json.dump(json_abstract, file, indent = 6) 
        return
    

In [55]:
query_tools = MyQuery(offsetYear = 0,dir = "output2")

Input your API keycc36e050bd46ac1513ace4dd80633bd3


In [ ]:
query_tools.search_loop()

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

In [10]:
dir = "output2"

In [11]:
def get_author(res):
    names = []
    if res["authors"] != None:
        for auth in res["authors"]["author"]:
            auth = auth["preferred-name"]
            names.append(auth["ce:indexed-name"])
    return ";".join(names)

def get_ref_title(res):
    title = []
    if res["item"]["bibrecord"]["tail"] != None:
        try:
            for ref in res["item"]["bibrecord"]["tail"]["bibliography"]["reference"]:
                    try:
                        if "ref-title" not in ref["ref-info"]:
                            continue
                        title.append(ref["ref-info"]["ref-title"]["ref-titletext"])
                    except:
                        pass
        except:
            pass
    return ";".join(title)

def get_ref_author(res):
    names = []
    if res["item"]["bibrecord"]["tail"] != None:
        try:
            for ref in res["item"]["bibrecord"]["tail"]["bibliography"]["reference"]:
                try:
                    if "ref-authors" not in ref["ref-info"]:
                        continue
                    if "author" not in ref["ref-info"]["ref-authors"]:
                        selection = ref["ref-info"]["ref-authors"]["collaboration"]
                        if type(selection) == list:
                            for sel in selection:
                                names.append(sel["ce:indexed-name"])
                        else:
                            names.append(selection["ce:indexed-name"])
                    else:
                        for auth in ref["ref-info"]["ref-authors"]["author"]:
                            names.append(auth["ce:indexed-name"])
                except:
                    pass
        except:
            pass
            
    return ";".join(list(set(filter(None,names))))
                

def extract_result(json_dict):
    result = {}
    res = json_dict['abstracts-retrieval-response']
    result["scopus_id"]= re.sub("SCOPUS_ID:","",res["coredata"]["dc:identifier"])
    result["author_name"] = get_author(res)
    if "dc:title" not in res["coredata"]:
        result["title"] = ""
    else:
        result["title"] = res["coredata"]["dc:title"]
    result["abstracts"] = res["coredata"]["dc:description"] if "dc:description" in res["coredata"] else None
    result["abstracts"] = "" if result["abstracts"]  == None else result["abstracts"]
    result["related_field"] = []
    result["auth_keywords"] = []
    date = res["item"]["bibrecord"]["head"]["source"]["publicationdate"]
    if "day" in date:
        day = date["day"]
    else:
        day = -1
    if "month" in date:
        month = date["month"]
    else:
        month = -1
    year = date["year"]
    result["publish_day"] =day
    result["publish_month"] =month
    result["publish_year"] =year
    if res["item"]["bibrecord"]["tail"] != None:
        result["ref_count"] = int(res["item"]["bibrecord"]["tail"]["bibliography"]["@refcount"])
    else:
        #print(res)
        result["ref_count"] = 0
        
    result["ref_authors"] = get_ref_author(res)
    result["ref_titles"] = get_ref_title(res)
    if "citedby-count" in res["coredata"]:
        if res["coredata"]["citedby-count"] != None:
            result["cited_count"] = int(res["coredata"]["citedby-count"])
        else:
            result["cited_count"] = 0
    else:
        result["cited_count"] = 0
    if (res["subject-areas"] != None):
        for area in res["subject-areas"]["subject-area"]:
            try:
                result["related_field"].append(area["@abbrev"])
            except Exception as e:
                print("@abbrev key not found")
    result["related_field"] = ";".join(result["related_field"])
    if (res["authkeywords"] != None):
        if type(res["authkeywords"]["author-keyword"]) == list:
            for keyword in res["authkeywords"]["author-keyword"]:
                try:
                    result["auth_keywords"].append(keyword["$"])
                except Exception as e:
                    print(res["authkeywords"]["author-keyword"])
                    print("$ key not found")
        else:
            result["auth_keywords"].append(res["authkeywords"]["author-keyword"]["$"])
    result["auth_keywords"] = ";".join(result["auth_keywords"])
    return result

In [12]:
data_point = glob.glob(f"{dir}/*")
data_point

['output2\\0035293021',
 'output2\\84924384215',
 'output2\\84929698635',
 'output2\\84951959696',
 'output2\\84958083769',
 'output2\\84975781055',
 'output2\\84988711922',
 'output2\\84997002173',
 'output2\\85000259359',
 'output2\\85005952241',
 'output2\\85006272627',
 'output2\\85008227599',
 'output2\\85008245198',
 'output2\\85009799127',
 'output2\\85012873655',
 'output2\\85012875280',
 'output2\\85013083057',
 'output2\\85014090638',
 'output2\\85015707925',
 'output2\\85015764556',
 'output2\\85016010727',
 'output2\\85017165971',
 'output2\\85018241471',
 'output2\\85018487453',
 'output2\\85018688325',
 'output2\\85019247372',
 'output2\\85019729838',
 'output2\\85020119118',
 'output2\\85020185630',
 'output2\\85020248364',
 'output2\\85020261089',
 'output2\\85020296213',
 'output2\\85020316170',
 'output2\\85020436220',
 'output2\\85020514930',
 'output2\\85021081527',
 'output2\\85021106763',
 'output2\\85021422600',
 'output2\\85021716674',
 'output2\\85022034986',
 

In [13]:
my_zipper = []
for path in tqdm(data_point):
    with open(path,'r') as file:
        jfile = json.load(file)
        result = extract_result(jfile)
        my_zipper.append((result["scopus_id"],result['author_name'],result['title'],result['abstracts'],result['related_field'],result['auth_keywords'],result['publish_day'],result['publish_month'],result['publish_year'],result['ref_count'],result["ref_authors"],result["ref_titles"],result['cited_count']))
                

  0%|          | 0/9600 [00:00<?, ?it/s]

In [14]:
col = ["scopus_id","author_name","title","abstracts","related_field","auth_keywords","publish_day","publish_month","publish_year","ref_count","ref_authors","ref_titles","cited_count"]
arr = list(zip(*my_zipper))
datapoint = {key : list(value) for key, value in zip(col,zip(*my_zipper))}

In [15]:
scrap_df = pd.DataFrame(datapoint)
scrap_df['scopus_id'] = scrap_df['scopus_id'].astype(str)
scrap_df = scrap_df.set_index('scopus_id')
scrap_df

,author_name,title,abstracts,related_field,auth_keywords,publish_day,publish_month,publish_year,ref_count,ref_authors,ref_titles,cited_count
scopus_id,,,,,,,,,,,,
0035293021,Baugh L.R.;Hill A.A.;Brown E.L.;Hunter C.P.,Quantitative analysis of mRNA amplification by...,Effective transcript profiling in animal syste...,BIOC,,01,03,2021,29,Riggins G.J.;Hudson J.;Erlander M.G.;Arnaud-Ba...,Exploring the metabolic and genetic control of...,424
84924384215,Bourdieu P.,Cultural reproduction and social reproduction,,SOCI,,11,05,2018,6,Praderie M.;Sofres;Dare;Bourdieu P.;de Saint-M...,Heritage social et chances d'ascension,853
84929698635,Wang G.;Deb S.;Cui Z.,Monarch butterfly optimization,"In nature, the eastern North American monarch ...",COMP;COMP,Benchmark problems;Butterfly adjusting operato...,01,07,2019,57,Coelho L.S.;Li J.;Brest J.;Yin M.;Fister I.;Ar...,Theory and applications of swarm intelligence;...,754
84951959696,Haider A.;Haider S.;Kang I.K.,A comprehensive review summarizing the effect ...,Nanotechnology is a budding technology that ha...,CHEM;CENG,Biomedical applications;Electrospinning;Parame...,01,12,2018,239,Walker J.;Huang C.;Chung C.-P.;Sohrabi A.;Li F...,Antimicrobial wound dressing nanofiber mats fr...,1155
84958083769,Blake G.R.;Hartge K.H.,Bulk density,Bulk density varies with structural condition ...,ENGI;AGRI,Archimedes’ principle;Bulk density;Clod method...,11,09,2018,12,Mintzer S.;Weaver H.H.;Reed I.F.;U. S. Departm...,Bulk density;Use of Saran Resin to coat natura...,3829
...,...,...,...,...,...,...,...,...,...,...,...,...
85190071715,Volk T.M.,"Music, Education, and Multiculturalism: Founda...","In today’s multi-ethnic classroom, multicultur...",ARTS,,01,01,2023,729,Olson J.S.;Nettl B.;Clark F.E.;Robinson J.T.;L...,Nationalizing Education;Changing Conceptions o...,107
85190074221,Fischer O.,Morphosyntactic Change: Functional and Formal ...,This book presents a critical comparison of th...,ARTS;SOCI,,01,01,2023,512,Horst van der J.M.;Zwart J.W.;Wray A.;Toorn M....,A lovely little example: Word order options an...,180
85190076689,Hudson R.,Language Networks: The New Word Grammar,This book argues that language is a network of...,ARTS;SOCI,,01,01,2023,302,Griffin R.;Saussure F.;Karlsson F.;Haspelmath ...,Valency and valency grammar;The representation...,207


In [16]:
scrap_df[scrap_df['cited_count'] < 200]

,author_name,title,abstracts,related_field,auth_keywords,publish_day,publish_month,publish_year,ref_count,ref_authors,ref_titles,cited_count
scopus_id,,,,,,,,,,,,
85073932524,Braun V.;Clarke V.;Hayfield N.,"‘A starting point for your journey, not a map’...",,PSYC,Codebook;coding;emerge;interpretation;methodol...,-1,-1,2022,37,Gray D.;Antaki C.;Spencer L.;Potter J.;Brooks ...,Thematic analysis;How to use thematic analysis...,186
85079074235,Kirtley J.;O'Mahony S.,What is a pivot? Explaining when and how entre...,Research Summary: Most theories of strategic c...,BUSI;BUSI,entrepreneurial firm strategy;innovation;pivot...,01,01,2023,108,Waters J.A.;Long C.P.;Rothaermel F.T.;Cui A.S....,Value creation in innovation ecosystems: How t...,104
85083155020,Samanta D.;Sivaram M.;Rashed A.N.Z.;Boopathi C...,Distributed Feedback Laser (DFB) for Signal Po...,This study outlines the distributed feedback l...,PHYS;PHYS;ENGI,DFB;hyperbolic secant pulse generator;optical ...,01,12,2023,18,Praveen Chakkravarthy S.;El Mashade M.B.;Tabou...,Bi-directional optical fiber transmission sche...,88
85083983060,Butt C.;Gill J.;Chun D.;Babu B.A.,RETRACTED ARTICLE: Deep learning system to scr...,The Editor in Chief has retracted this article...,COMP,,01,02,2023,8,,,101
85084131054,Choi T.M.,Supply chain financing using blockchain: impac...,"Today, supply chain finance is a very importan...",DECI;DECI,Blockchain;Coordination;Fashionable products;M...,01,12,2023,46,Yang S.A.;Li R.;Wang A.;Buzacott J.;Shi X.;Car...,Risk intermediation in supply chains;Data anal...,106
...,...,...,...,...,...,...,...,...,...,...,...,...
85190011748,Dixon R.M.W.;Vogel A.R.,The Jarawara Language of Southern Amazonia,"This is the first account of Jarawara, a South...",ARTS;SOCI,,01,01,2023,81,Marlett S.;Da Silva E.P.;Merlan F.;Ehrenreich ...,Madija noun morphology;Antipasivo en madija (c...,97
85190024946,Aarts B.,Syntactic Gradience: The Nature of Grammatical...,This is the first exhaustive investigation of ...,ARTS;SOCI,,01,01,2023,451,Fried M.;Schane S.A.;Fodor J.A.;Bjarkman P.C.;...,Verb-preposition constructions and small claus...,129
85190041246,Gussmann E.,THE PHONOLOGY OF POLISH,This book is the most complete phonology of co...,ARTS;SOCI,,01,01,2023,234,Kiparsky P.;Puppel S.;Szigetvaeri P.;Gorska E....,A study in diachronic morphophonemics: Ukraini...,132


In [17]:
scrap_df.to_csv("scrap_df2.csv")

In [18]:
rd = redis.Redis(host = "107.22.67.15",port = 6379,password = "cryogen",charset="utf-8", decode_responses=True)
rd.ping()

True

In [19]:
cache = set([re.sub("scopus:","",key) for key in rd.keys()])
cache

{'85127478185',
 '85127426918',
 '85191156615',
 '85164101539',
 '85185668220',
 '85112325822',
 '85071868943',
 '85129802632',
 '85127394878',
 '85188511579',
 '85119566664',
 '85129818144',
 '85177670679',
 '85105274575',
 '85145341600',
 '85149837200',
 '85082779722',
 '85118475949',
 '85146232467',
 '85062239217',
 '85185514930',
 '85187157733',
 '85151234904',
 '85107307321',
 '85167914543',
 '85080134686',
 '85128079841',
 '85112339653',
 '85173096294',
 '85169591908',
 '85080023680',
 '85107231199',
 '85152067380',
 '85101534884',
 '85112248022',
 '85127809460',
 '85182088509',
 '85061837862',
 '85146248046',
 '85061791529',
 '85100690333',
 '85083959994',
 '85188257021',
 '85061837391',
 '85063094842',
 '85147824102',
 '85185253385',
 '85112246717',
 '85103274361',
 '85076208414',
 '85111178857',
 '85187244777',
 '85177676285',
 '85172794707',
 '85079321926',
 '85145515460',
 '85183415291',
 '85100177574',
 '85186500833',
 '85170386039',
 '85150931828',
 '85128094918',
 '851598

In [20]:
col = ["scopus_id","author_name","title","abstracts","related_field","auth_keywords","publish_day","publish_month","publish_year","ref_count","ref_authors","ref_titles","cited_count"]

In [21]:
def multiThreadSender(thread_size = 8):
    cache = set([re.sub("scopus:","",key) for key in rd.keys()])
    scrape_ids = np.array(list(set(scrap_df.index.values) - cache))
    choice = np.random.randint(0,thread_size,size = scrape_ids.shape)
    my_thread = [None] * thread_size
    scrape_arr = [[]] * thread_size
    
    for i in range(thread_size):
        scrape_arr[i] = scrape_ids[choice == i].tolist()
    
    def threadSender(scrape_id,isTqdm):
        iterator = None
        if (isTqdm):
            iterator = tqdm(scrape_id)
        else:
            iterator = scrape_id
        for scopus_id in iterator:
            row = scrap_df.loc[scopus_id]
            my_map = {c : int(row[c]) if (c == "ref_count" or c == "cited_count") else row[c] for c in col[1:]}
            rd.hset(f"scopus:{scopus_id}",mapping = my_map)
    for i in range(thread_size):
        my_thread[i] = threading.Thread(target=threadSender,args = (scrape_arr[i],i == 0))
        my_thread[i].start()
    
    for i in tqdm(range(thread_size)):
        my_thread[i].join()
    
    return 

In [22]:
multiThreadSender(2048)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2048 [00:00<?, ?it/s]

In [23]:
len(rd.keys())

19201

In [27]:
rd.hgetall("scopus:85082952135")

{'author_name': 'Gorjanc V.;Stabej M.',
 'title': 'Slovenščina 2.0: »jezik in družbeni spol',
 'abstracts': '',
 'related_field': 'ARTS;SOCI',
 'auth_keywords': '',
 'publish_day': '31',
 'publish_month': '12',
 'publish_year': '2019',
 'ref_count': '0',
 'ref_authors': '',
 'ref_titles': '',
 'cited_count': '0'}